In [1]:
#STEP 3.2 - Data imputation & Encoding
'''
In this notebook we are going to handle missing data, by imputing missing data via median/mode..
also, we are going to encode via one-hot encoding or dummy encoding  variable so that the predictive model can process and use them.
'''

In [6]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt

In [7]:
#now to data imputation

df = pd.read_pickle('df_cleaned_with_outliers.pkl')
df.head()

,ID,Source,Severity,Start_Time,End_Time,Start_Lat,Start_Lng,End_Lat,End_Lng,Distance(mi),...,End_Year,End_Month,End_Day,End_DayOfWeek,End_Hour,End_Minute,End_Second,End_IsWeekend,Duration_Minutes,Duration_Hours
0,A1,Source2,3,2016-02-08 05:46:00,2016-02-08 11:00:00,39.865147,-84.058723,NaN,NaN,0.01,...,2016.0,2.0,8.0,0.0,11.0,0.0,0.0,0,314.0,5.233333
1,A2,Source2,2,2016-02-08 06:07:59,2016-02-08 06:37:59,39.928059,-82.831184,NaN,NaN,0.01,...,2016.0,2.0,8.0,0.0,6.0,37.0,59.0,0,30.0,0.500000
2,A3,Source2,2,2016-02-08 06:49:27,2016-02-08 07:19:27,39.063148,-84.032608,NaN,NaN,0.01,...,2016.0,2.0,8.0,0.0,7.0,19.0,27.0,0,30.0,0.500000
3,A4,Source2,3,2016-02-08 07:23:34,2016-02-08 07:53:34,39.747753,-84.205582,NaN,NaN,0.01,...,2016.0,2.0,8.0,0.0,7.0,53.0,34.0,0,30.0,0.500000
4,A5,Source2,2,2016-02-08 07:39:07,2016-02-08 08:09:07,39.627781,-84.188354,NaN,NaN,0.01,...,2016.0,2.0,8.0,0.0,8.0,9.0,7.0,0,30.0,0.500000


In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7728394 entries, 0 to 7728393
Data columns (total 64 columns):
 #   Column                 Dtype         
---  ------                 -----         
 0   ID                     string        
 1   Source                 category      
 2   Severity               category      
 3   Start_Time             datetime64[ns]
 4   End_Time               datetime64[ns]
 5   Start_Lat              float64       
 6   Start_Lng              float64       
 7   End_Lat                float64       
 8   End_Lng                float64       
 9   Distance(mi)           float64       
 10  Description            string        
 11  Street                 string        
 12  City                   string        
 13  County                 string        
 14  State                  string        
 15  Zipcode                string        
 16  Country                string        
 17  Timezone               string        
 18  Airport_Code          

In [9]:
print(df.isna().sum().sort_values())

ID                         0
Amenity                    0
Bump                       0
Give_Way                   0
Junction                   0
                      ...   
Duration_Hours        743166
Wind_Chill(F)        1999019
Precipitation(in)    2203586
End_Lat              3402762
End_Lng              3402762
Length: 64, dtype: int64


In [10]:
missing_summary = df.isnull().sum().sort_values(ascending=False)
missing_summary = missing_summary[missing_summary > 0]
print(missing_summary)


End_Lat                  3402762
End_Lng                  3402762
Precipitation(in)        2203586
Wind_Chill(F)            1999019
Duration_Hours            743166
End_Month                 743166
Start_Day                 743166
Start_Hour                743166
Start_Minute              743166
Start_Second              743166
Start_Month               743166
Start_DayOfWeek           743166
End_Year                  743166
End_Day                   743166
End_DayOfWeek             743166
End_Hour                  743166
End_Minute                743166
End_Second                743166
Start_Year                743166
End_Time                  743166
Start_Time                743166
Duration_Minutes          743166
Wind_Speed(mph)           571233
Visibility(mi)            177098
Wind_Direction            175206
Humidity(%)               174144
Weather_Condition         173459
Temperature(F)            163853
Pressure(in)              140679
Weather_Timestamp         120228
Astronomic

In [11]:
#=========================================
# US Accidents Hybrid Imputation Pipeline (Final Version)
# Target: Duration_Minutes / Duration_Hours
#=========================================
import pandas as pd
import numpy as np
from sklearn.impute import SimpleImputer, KNNImputer
from sklearn.preprocessing import StandardScaler

#=========================================
# 1. HANDLE HEAVY-MISSING COORDINATES
#=========================================
# >40% missing → cannot impute accurately
# Option 1: fill with start coordinates (approximation)
df['End_Lat'] = df['End_Lat'].fillna(df['Start_Lat'])
df['End_Lng'] = df['End_Lng'].fillna(df['Start_Lng'])
# Option 2: drop columns if not used later
# df = df.drop(columns=['End_Lat', 'End_Lng'])

#=========================================
# 2. IMPUTE WEATHER FEATURES
#=========================================
weather_cols_all = [
    'Temperature(F)', 'Humidity(%)', 'Pressure(in)',
    'Visibility(mi)', 'Wind_Speed(mph)', 'Precipitation(in)', 'Wind_Chill(F)'
]

# --- Step 2a: Simple groupwise median ---
for col in weather_cols_all:
    df[col] = df.groupby('State')[col].transform(lambda x: x.fillna(x.median()))

# --- Step 2b: Selective KNN for highly-missing correlated numeric features (>10% missing) ---
weather_for_knn = ['Temperature(F)', 'Humidity(%)', 'Pressure(in)', 'Visibility(mi)', 'Wind_Speed(mph)']
missing_ratio = df[weather_for_knn].isna().mean()
cols_for_knn = missing_ratio[missing_ratio > 0.10].index.tolist()

if cols_for_knn:
    print(f"⚙️ Applying KNN imputation to: {cols_for_knn}")
    scaler = StandardScaler()
    scaled = scaler.fit_transform(df[cols_for_knn])
    knn = KNNImputer(n_neighbors=3, weights='uniform')
    imputed_scaled = knn.fit_transform(scaled)
    df[cols_for_knn] = scaler.inverse_transform(imputed_scaled)
else:
    print("✅ No need for KNN imputation (all weather missingness <10%).")

#=========================================
# 3. CATEGORICAL FEATURES
#=========================================
cat_cols = [
    'Wind_Direction', 'Weather_Condition',
    'Sunrise_Sunset', 'Civil_Twilight',
    'Nautical_Twilight', 'Astronomical_Twilight'
]

for col in cat_cols:
    if df[col].isna().mean() > 0:
        df[col] = df.groupby('State')[col].transform(
            lambda x: x.fillna(x.mode().iloc[0]) if not x.mode().empty else x
        )

#=========================================
# 4. WEATHER TIMESTAMP
#=========================================
df = df.sort_values(['City', 'Start_Time'])
df['Weather_Timestamp'] = df.groupby('City')['Weather_Timestamp'].ffill().bfill()

#=========================================
# 5. HANDLE DATETIME & TARGET (Duration) FIELDS
#=========================================
# Drop rows missing both Start_Time and End_Time
df = df.dropna(subset=['Start_Time', 'End_Time'])

# Recompute duration where both timestamps exist but Duration is missing
mask = df['Duration_Minutes'].isna() & df['Start_Time'].notna() & df['End_Time'].notna()
df.loc[mask, 'Duration_Minutes'] = (df.loc[mask, 'End_Time'] - df.loc[mask, 'Start_Time']).dt.total_seconds() / 60
df.loc[mask, 'Duration_Hours'] = df.loc[mask, 'Duration_Minutes'] / 60

# Drop any rows still missing target variable
df = df.dropna(subset=['Duration_Minutes', 'Duration_Hours'])

#=========================================
# 6. RECOMPUTE DERIVED TIME FEATURES
#=========================================
df['Start_Year'] = df['Start_Time'].dt.year
df['Start_Month'] = df['Start_Time'].dt.month
df['Start_Day'] = df['Start_Time'].dt.day
df['Start_DayOfWeek'] = df['Start_Time'].dt.dayofweek
df['Start_Hour'] = df['Start_Time'].dt.hour
df['Start_Minute'] = df['Start_Time'].dt.minute
df['Start_Second'] = df['Start_Time'].dt.second
df['Start_IsWeekend'] = (df['Start_DayOfWeek'] >= 5).astype(int)

df['End_Year'] = df['End_Time'].dt.year
df['End_Month'] = df['End_Time'].dt.month
df['End_Day'] = df['End_Time'].dt.day
df['End_DayOfWeek'] = df['End_Time'].dt.dayofweek
df['End_Hour'] = df['End_Time'].dt.hour
df['End_Minute'] = df['End_Time'].dt.minute
df['End_Second'] = df['End_Time'].dt.second
df['End_IsWeekend'] = (df['End_DayOfWeek'] >= 5).astype(int)

#=========================================
# 7. FINAL IMPUTATION PASS (Predictors only)
#=========================================
# Numeric predictors
num_cols = [c for c in df.select_dtypes(include=['float64', 'int64']).columns
            if c not in ['Duration_Minutes', 'Duration_Hours']]  # exclude targets

imputer = SimpleImputer(strategy='median')
df[num_cols] = imputer.fit_transform(df[num_cols])

# Categorical predictors
cat_cols_all = df.select_dtypes(include=['category', 'object', 'string']).columns
for col in cat_cols_all:
    if df[col].isna().sum() > 0:
        df[col] = df[col].fillna(df[col].mode()[0])

#=========================================
# 8. FINAL CHECK
#=========================================
missing_summary = df.isna().sum().sort_values(ascending=False)
print("✅ Hybrid imputation complete (target-safe).")
print("Remaining missing values:\n", missing_summary[missing_summary > 0])


✅ No need for KNN imputation (all weather missingness <10%).
✅ Hybrid imputation complete (target-safe).
Remaining missing values:
 Series([], dtype: int64)


In [12]:
df.isna().sum()

ID                  0
Source              0
Severity            0
Start_Time          0
End_Time            0
                   ..
End_Minute          0
End_Second          0
End_IsWeekend       0
Duration_Minutes    0
Duration_Hours      0
Length: 64, dtype: int64

In [14]:
'''
Let’s build a clean, automated encoding pipeline that:

Selects all categorical columns from your DataFrame.

Counts the number of unique categories in each.

Applies:

One-Hot Encoding for binary (2-category) columns.

Dummy Encoding (OHE with drop_first=True) for columns with >2 categories.

Combines all encoded features into a new DataFrame encoded_df.

Compares original vs encoded columns and identifies reference categories (the one dropped in dummy encoding).
'''
def encode_categorical_features_safe(df, max_unique=20):
    """
    Safely encode categorical features in a DataFrame.

    Rules:
    - Skips columns with '_' in the name (engineered features)
    - Skips columns with only 0–1 unique values
    - Skips columns with > max_unique categories (too large for OHE)
    - One-hot encodes binary categorical columns (2 unique values)
    - Dummy encodes multi-class categorical columns (>2 and <= max_unique)
    
    Returns:
        encoded_df: encoded categorical features (numeric)
        ref_df: reference summary with dropped categories
    """
    df = df.copy()

    encoded_dfs = []
    reference_summary = []

    # 1️⃣ Select categorical and string columns
    cat_cols = df.select_dtypes(include=['category', 'object', 'string']).columns
    cat_cols = [col for col in cat_cols if '_' not in col]

    if len(cat_cols) == 0:
        print("⚠️ No categorical columns found for encoding.")
        return pd.DataFrame(), pd.DataFrame()

    # 2️⃣ Iterate columns
    for col in cat_cols:
        n_unique = df[col].nunique(dropna=True)
        if n_unique <= 1:
            reference_summary.append({
                'Column': col, 'Unique_Categories': n_unique,
                'Encoding_Type': 'Skipped (constant)', 'Reference_Category': None
            })
            continue

        elif n_unique > max_unique:
            print(f"⚠️ Skipping '{col}' ({n_unique} categories) — too many unique values")
            reference_summary.append({
                'Column': col, 'Unique_Categories': n_unique,
                'Encoding_Type': 'Skipped (high-cardinality)', 'Reference_Category': None
            })
            continue

        # One-hot or dummy encoding
        if n_unique == 2:
            encoded = pd.get_dummies(df[col], prefix=col, drop_first=False)
            ref_value = None
            encoding_type = 'One-Hot'
        else:
            encoded = pd.get_dummies(df[col], prefix=col, drop_first=True)
            ref_value = sorted(df[col].dropna().unique())[0]
            encoding_type = 'Dummy'

        encoded_dfs.append(encoded)
        reference_summary.append({
            'Column': col,
            'Unique_Categories': n_unique,
            'Encoding_Type': encoding_type,
            'Reference_Category': ref_value
        })

    # 3️⃣ Combine encoded features
    encoded_df = pd.concat(encoded_dfs, axis=1) if encoded_dfs else pd.DataFrame()
    ref_df = pd.DataFrame(reference_summary)

    print(f"✅ Encoded {len(encoded_dfs)} categorical columns successfully.")
    return encoded_df, ref_df


# Example usage
encoded_df, ref_df = encode_categorical_features_safe(df, max_unique=20)

# Keep numeric / boolean columns
non_cat_df = df.select_dtypes(exclude=['category', 'object', 'string'])

# Merge into final modeling dataset
final_df = pd.concat([non_cat_df, encoded_df], axis=1)

print(f"✅ Final dataset shape: {final_df.shape}")
final_df.head()


⚠️ Skipping 'ID' (6985228 categories) — too many unique values
⚠️ Skipping 'Description' (3542610 categories) — too many unique values
⚠️ Skipping 'Street' (319545 categories) — too many unique values
⚠️ Skipping 'City' (13593 categories) — too many unique values
⚠️ Skipping 'County' (1822 categories) — too many unique values
⚠️ Skipping 'State' (49 categories) — too many unique values
⚠️ Skipping 'Zipcode' (771885 categories) — too many unique values
✅ Encoded 3 categorical columns successfully.
✅ Final dataset shape: (6985228, 55)


,Start_Time,End_Time,Start_Lat,Start_Lng,End_Lat,End_Lng,Distance(mi),Weather_Timestamp,Temperature(F),Wind_Chill(F),...,Duration_Hours,Source_Source2,Source_Source3,Severity_2,Severity_3,Severity_4,Timezone_USEastern,Timezone_USMountain,Timezone_USPacific,Timezone_Unknown
2796054,2018-05-06 19:08:26,2018-05-06 19:38:12,40.905312,-77.421593,40.905312,-77.421593,0.000,2018-05-06 18:53:00,51.8,48.0,...,0.496111,True,False,True,False,False,True,False,False,False
2390565,2018-12-17 07:50:05,2018-12-17 08:19:12,40.883820,-77.467682,40.883820,-77.467682,0.000,2018-12-17 06:53:00,35.6,29.2,...,0.485278,True,False,True,False,False,True,False,False,False
7718870,2019-08-04 00:09:11,2019-08-04 00:38:29,40.905501,-77.419572,40.904403,-77.401810,0.931,2019-08-04 00:16:00,67.0,67.0,...,0.488333,False,False,False,True,False,True,False,False,False
7718871,2019-08-04 00:09:11,2019-08-04 00:38:29,40.904403,-77.401810,40.905501,-77.419572,0.931,2019-08-04 00:16:00,67.0,67.0,...,0.488333,False,False,False,True,False,True,False,False,False
1402241,2020-08-05 18:18:34,2020-08-05 21:54:51,40.904720,-77.388947,40.904720,-77.388947,0.000,2020-08-05 17:53:00,81.0,81.0,...,3.604722,True,False,True,False,False,True,False,False,False


In [15]:
ref_df


,Column,Unique_Categories,Encoding_Type,Reference_Category
0,ID,6985228,Skipped (high-cardinality),None
1,Source,3,Dummy,Source1
2,Severity,4,Dummy,1
3,Description,3542610,Skipped (high-cardinality),None
4,Street,319545,Skipped (high-cardinality),None
5,City,13593,Skipped (high-cardinality),None
6,County,1822,Skipped (high-cardinality),None
7,State,49,Skipped (high-cardinality),None
8,Zipcode,771885,Skipped (high-cardinality),None
9,Country,1,Skipped (constant),None


In [16]:
final_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 6985228 entries, 2796054 to 3679722
Data columns (total 55 columns):
 #   Column               Dtype         
---  ------               -----         
 0   Start_Time           datetime64[ns]
 1   End_Time             datetime64[ns]
 2   Start_Lat            float64       
 3   Start_Lng            float64       
 4   End_Lat              float64       
 5   End_Lng              float64       
 6   Distance(mi)         float64       
 7   Weather_Timestamp    datetime64[ns]
 8   Temperature(F)       float64       
 9   Wind_Chill(F)        float64       
 10  Humidity(%)          float64       
 11  Pressure(in)         float64       
 12  Visibility(mi)       float64       
 13  Wind_Speed(mph)      float64       
 14  Precipitation(in)    float64       
 15  Amenity              bool          
 16  Bump                 bool          
 17  Crossing             bool          
 18  Give_Way             bool          
 19  Junction            

In [17]:
#save to pickle 

final_df.to_pickle("final_df.pkl")